In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import tensorflow as tf
import requests
from PIL import Image
import matplotlib.pyplot as plt

Let's start by including the code that gets the data and model:

In [2]:
!wget -Nq https://raw.githubusercontent.com/MicrosoftDocs/tensorflow-learning-path/main/intro-tf/tintro.py
from tintro import *

In order to make a prediction, you need to pass some data to the model, and do a single forward pass through the network to get the prediction. If the code below is unclear to you, make sure you go back to module 1, where we explain it in detail. Remember that, unlike during testing, you don't need to call the loss function because you're no longer interested in evaluating how well the model is doing. Instead, you call `softmax` to convert the values of the output vector into values between 0 and 1, and then get the `argmax` of that vector to get the predicted label index.

Similarly to the training and testing sections, once you're done with debugging, you can add a `@tf.function` decorator to get the performance benefits of graph execution.

In [3]:
@tf.function
def predict(model: tf.keras.Model, X: np.ndarray) -> tf.Tensor:
  y_prime = model(X, training=False)
  probabilities = tf.nn.softmax(y_prime, axis=1)
  predicted_indices = tf.math.argmax(input=probabilities, axis=1)
  return predicted_indices

You can now make a prediction. First you get the image you'll use for prediction.

In [ ]:
model = NeuralNetwork()
model.load_weights('outputs/weights')

url = 'https://raw.githubusercontent.com/MicrosoftDocs/tensorflow-learning-path/main/intro-keras/predict-image.png'

with Image.open(requests.get(url, stream=True).raw) as image:
  X = np.asarray(image, dtype=np.float32).reshape((-1, 28, 28)) / 255.0

plt.figure()
plt.axis('off')
plt.imshow(X.squeeze(), cmap='gray')
plt.show()

You're now ready to make the prediction by passing the image you read into the `predict` function.

In [ ]:
predicted_index = predict(model, X).numpy()[0]
predicted_name = labels_map[predicted_index]

print(f'Predicted class: {predicted_name}')